# Preprocesamiento

El siguiente bloque es un resumen de nuestro ETL, cuando tengamos la data final, lo quitaremos

In [1]:
### SCRIPT DE obtencion de datos
#### Este script lo vamos a quitar pues todo lo que hace es que resume el ETL para obtener los datos

import pandas as pd
import yaml
import numpy as np

### User defined
import variables_n_functions as vnf

config_file = open('config.yaml', 'r')
config = yaml.safe_load(config_file)

teams = config['teams']


### We initialize the df with the appropriate column names
df = pd.DataFrame(columns = vnf.columnas_df)

### The following variable is auxiliar to avoid duplicate requests
teams_aux = list(teams.keys())

### We recover the match history between every unique team - team combination, and store it in the df
for team_1 in teams.keys():
    teams_aux.remove(team_1)
    for team_2 in teams_aux:
        h2h = vnf.head2head(team_1, team_2, config['sports_token'])
        df = pd.concat([df] + [pd.DataFrame(pd.Series(h2h[k])).transpose() for k in range(len(h2h))])        

### Define columns to drop or that will be added to other tables
dropped_columns = ['details']
to_other_tables = ['weather_report', 'formations', 'scores', 'time', 'coaches', 'standings', 'assistants', 'colors']

config_file = open('config.yaml', 'r')
config = yaml.safe_load(config_file)

df_general = df.copy().drop(dropped_columns + to_other_tables, 1)

df_scores = pd.DataFrame(columns = ['id', 'localteam_score', 'visitorteam_score', 'localteam_pen_score',
                                    'visitorteam_pen_score', 'ht_score', 'ft_score', 'et_score', 'ps_score'])
for k in range(df.shape[0]):
#     temp = pd.DataFrame({key:[value] for key,value in eval(df['scores'].iloc[k]).items()})
    temp = pd.DataFrame({key:[value] for key,value in df['scores'].iloc[k].items()})
    temp['id'] = df.iloc[k]['id']
    df_scores = pd.concat([df_scores, temp])
    
#################### Transformations ####################
    
###### h2h.general

### ID variables
df_general['id'] = df_general['id']
df_general['league_id'] = df_general['league_id'].fillna(-1)
df_general['season_id'] = df_general['season_id'].fillna(-1)
df_general['stage_id'] = df_general['stage_id'].fillna(-1)
df_general['round_id'] = df_general['round_id'].fillna(-1)
df_general['group_id'] = df_general['group_id'].fillna(-1)
df_general['aggregate_id'] = df_general['aggregate_id'].fillna(-1)
df_general['venue_id'] = df_general['venue_id'].fillna(-1)
df_general['referee_id'] = df_general['referee_id'].fillna(-1)
df_general['localteam_id'] = df_general['localteam_id'].fillna(-1)
df_general['visitorteam_id'] = df_general['visitorteam_id'].fillna(-1)
df_general['winner_team_id'] = df_general['winner_team_id'].fillna(-1)

### Other variables
df_general['commentaries'] = df_general['commentaries'].apply(vnf.booleanize)# Boolean
df_general['attendance'] = df_general['attendance'].fillna(-1)# Integer
df_general['pitch'] = df_general['pitch'].apply(lambda x: "None" if x is None else x) # Categorical
df_general['neutral_venue'] = df_general['neutral_venue'].apply(vnf.booleanize)# Boolean
df_general['winning_odds_calculated'] = df_general['winning_odds_calculated'].apply(vnf.booleanize)# Boolean
df_general['deleted'] = df_general['deleted'].apply(vnf.booleanize)# Boolean
df_general['is_placeholder'] = df_general['is_placeholder'].apply(vnf.booleanize)# Boolean
df_general['leg'] = df_general['leg'].fillna(-1)

###### h2h.scores

df_scores['id'] = df_scores['id']
df_scores['localteam_score'] = df_scores['localteam_score'].fillna(-1)# Integer
df_scores['visitorteam_score'] = df_scores['visitorteam_score'].fillna(-1)# Integer
df_scores['localteam_pen_score'] = df_scores['localteam_pen_score'].fillna(-1)# Integer
df_scores['visitorteam_pen_score'] = df_scores['visitorteam_pen_score'].fillna(-1)# Integer
df_scores['ht_score'] = df_scores['ht_score'].fillna(-1) # String
df_scores['ft_score'] = df_scores['ft_score'].fillna(-1) # String
df_scores['et_score'] = df_scores['et_score'].fillna(-1) # String
df_scores['ps_score'] = df_scores['ps_score'].fillna(-1) # String



<ipython-input-1-c26fcd96006e>:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_general = df.copy().drop(dropped_columns + to_other_tables, 1)


# Preprocesamiento

El siguiente bloque es el preprocesamiento pero a diferencia del anterior, aquí manipue el dataframe producto del etl para darle el formato adecuado para correr los modelos de manera muy sencilla. Esto incluye:
+ solo seleccionar las variables que usaremos por lo pronto
+ dar el formato adecuado al data frame
+ proponer las distintas Ys

In [3]:
# Defino mis datos y mi Y

#Creo variable 
# Agrego indice que en el ETL no lo conservamos
new_index = [i for i in range(len(df_general))]
df_general.index = new_index

# Creo la variable Y que en este caso es simplemente si el equipo local ganó (1) vs empato o perdió (0)
df_general['Y'] = np.where(df_general['winner_team_id']==df_general['localteam_id'], 1, 0)

# Aqui selecciono simplemente las variables que le voy a meter
dat=df_general[["league_id","season_id","venue_id","referee_id","localteam_id",'visitorteam_id']]

#dat1=pd.json_normalize(df["formations"])
#dat2=pd.json_normalize(df["scores"])
#dat3=pd.json_normalize(df["time"])
#dat4=pd.json_normalize(df["coaches"])
dat5=pd.json_normalize(df["standings"])
#dat6=pd.json_normalize(df["assistants"])
data=pd.concat([dat,dat5], axis=1)
X = pd.get_dummies(data)

# este es el tratamiento ingenuo a los NAs e infinitos
X=X.fillna(0) 
X = X.replace([np.inf, -np.inf], np.nan)
X = X.reset_index()

In [52]:
# Defino mis posibles variables Y
# Creo la variable Y que en este caso es simplemente si el equipo local ganó (1) vs empato o perdió (0)
y = df_general['Y']

# Otra es la cantidad de goles locales y la cantidad de goles visitantes, para simular resultados mas que probabilidad de ganar
df_scores.index = new_index
y_goals_local = df_scores['localteam_score']
y_goals_visitor = df_scores['visitorteam_score']

# Finalmente, otra opcion es crear categorías de empate, ganó o perdio, de esta manera podemos predecir también el empate por separado
y_multi = np.where(df_general['winner_team_id']==-1, "Empate", 
                  np.where(df_general['winner_team_id']==df_general['localteam_id'], "Local", "Visitante"))



Como pueden observar, los datos son muy sencillos, y con base en ellos los modelos lograron superar el benchmark inicial de vencer al azar.

In [77]:
X

,index,league_id,season_id,venue_id,referee_id,localteam_id,visitorteam_id,localteam_position,visitorteam_position
0,0,501,18369,8914,14859,62,53,2.0,1.0
1,1,501,18369,8909,14853,53,62,2.0,1.0
2,2,501,18369,8914,18748,62,53,6.0,5.0
3,3,501,17141,8914,14468,62,53,1.0,2.0
4,4,501,17141,8909,14859,53,62,2.0,1.0
...,...,...,...,...,...,...,...,...,...
2086,2086,501,1932,219,17252,734,496,0.0,0.0
2087,2087,501,1931,219,70310,734,496,0.0,0.0
2088,2088,501,1931,281425,70311,496,734,0.0,0.0
2089,2089,501,1931,219,70311,734,496,0.0,0.0


# Modelado

## CV

Como pueden ver, tenemos varias opciones de Ys, pero para que la comparacion de modelos sea justa usaremos la misma division y las mismas métricas: cantidad de partidos predecidos con exito.

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, y_local_train, y_local_test,y_visitor_train, y_visitor_test = train_test_split(X,y, y_goals_local,y_goals_visitor, test_size=0.4, random_state=10)


## Modelo 1: Lasso con Y binaria

Este es el modelo más sencillo donde solo se usa la Y binaria  (1 si se gana local vs 0 en otro caso)

In [83]:
# MODELO !

# Lasso BINARY MODEL

from sklearn.linear_model import Lasso

reg = Lasso(alpha=1)
reg.fit(X_train, y_train)
print('R squared training set', round(reg.score(X_train, y_train)*100, 2))
print('R squared test set', round(reg.score(X_test, y_test)*100, 2))

xtrain=reg.predict(X_train)
xtest=reg.predict(X_test)
partidos_train=sum(np.rint(np.nextafter(xtrain, xtrain+1))==y_train)/len(y_train)
partidos_test=sum(np.rint(np.nextafter(xtest, xtest+1))==y_test)/len(y_test)
print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))


R squared training set 6.27
R squared test set 4.53
% de partidos predecidos en train 63.48
% de partidos predecidos en test 61.29


## Modelo 2:  Lasso con Y numérica

Esta es una pareja de modelos que en lugar de usar la Y binaria usan como Y la cantidad de goles anotados por el local y el visitante, respectivamente. El diseño consiste en estimar un modelo lineal para estimar los goles de cada uno de los dos equipos y posteriormente, comparamos los goles del equipo 1 frente a los del equipo 2. Esto nos permite hacer una estimación del resultado y no solo determinar el ganador. Esto es importante porque podriamos rescatar la categoría de "Empate".

Sabemos que probablemente existan problemas de correlación entre la Y local y la Y visitante, por lo que no tiene mucho sentido matemático que hayamos estimado dos modelos de manera independiente, pero lo realizamos como un experimento. Sorprendentemente dio buenos resultados.

Podemos explorar el cómo hacer este tipo de modelos de manera correcta a futuro





In [84]:
# Modelo 2

# Mismo modelo anterior pero para predecir resultado completo

reg2 = Lasso(alpha=1)
model_local=reg2.fit(X_train, y_local_train)

print('R squared training set', round(reg2.score(X_train, y_local_train)*100, 2))
print('R squared test set', round(reg2.score(X_test, y_local_test)*100, 2))

reg3 = Lasso(alpha=1)
model_visitor=reg3.fit(X_train, y_visitor_train)
print('R squared training set', round(reg3.score(X_train, y_visitor_train)*100, 2))
print('R squared test set', round(reg3.score(X_test, y_visitor_test)*100, 2))


partidos_test=sum((np.round(model_local.predict(X_test)) > np.round(model_visitor.predict(X_test)))==y_test)/len(y_test)
partidos_train=sum((np.round(model_local.predict(X_train)) > np.round(model_visitor.predict(X_train)))==y_train)/len(y_train)
print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))




R squared training set 6.78
R squared test set 7.44
R squared training set 4.92
R squared test set 5.13
% de partidos predecidos en train 60.85
% de partidos predecidos en test 58.18


## Modelo 2.5:  Extension Lasso con Y numérica pero rescatando la categoría empate

Como el modelo anterior nos permite estimar los goles esperados, podemos proponer una alpha de tal tamaño que nos sirva para declarar empate si los goles esperados son similares entre ambos equipos.

Por ejemplo, si la alpha es de tamaño .11, y si los goles esperados son 1.95 y 2.05, para el local y visitante, pues estariamos proponiendo un empate de 2-2.

Notamos que la cantidad de partidos acertados disminuye sin embargo, ahora estamos hablando de un problema de 3 cateogrías, por lo que el benchmark a vencer sería un porcentaje de 1/3 de los partidos. Desde ese enfoque, el modelo tiene un buen desempeño.

In [86]:
# Evaluacion con empate!
indices_train=y_train.index[0:len(y_train)]
indices_test=y_test.index[0:len(y_test)]
pd_y_multi=pd.DataFrame(y_multi)
pd_y_multi.index = new_index
y_multi_train = pd_y_multi.filter(items = indices_train, axis=0)
y_multi_test = pd_y_multi.filter(items = indices_test, axis=0)

alpha = .05

predict_multi_train = np.where(model_local.predict(X_train) > alpha + (model_visitor.predict(X_train)), "Local", 
                  np.where(model_local.predict(X_train) + alpha < model_visitor.predict(X_train), "Visitante", "Empate"))

predict_multi_test = np.where(model_local.predict(X_test) > alpha + (model_visitor.predict(X_test)), "Local", 
                  np.where(model_local.predict(X_test) + alpha < model_visitor.predict(X_test), "Visitante", "Empate"))

partidos_train=sum(y_multi_train[0]==predict_multi_train)/len(predict_multi_train)
partidos_test=sum(y_multi_test[0]==predict_multi_test)/len(predict_multi_test)

print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))


% de partidos predecidos en train 47.05
% de partidos predecidos en test 47.19


## Modelo 4:  Lasso CV

Es una version similar a las anteriores con Y binaria.


In [88]:
# Modelo 3 LASSO CV

from sklearn.linear_model import LassoCV

# Lasso with 5 fold cross-validation
model = LassoCV(cv=5, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

# Set best alpha
lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)
print('R squared training set', round(lasso_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_best.score(X_test, y_test)*100, 2))

xtrain=lasso_best.predict(X_train)
xtest=lasso_best.predict(X_test)
partidos_train=sum(np.rint(np.nextafter(xtrain, xtrain+1))==y_train)/len(y_train)
partidos_test=sum(np.rint(np.nextafter(xtest, xtest+1))==y_test)/len(y_test)
print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))


R squared training set 5.74
R squared test set 4.17
% de partidos predecidos en train 63.96
% de partidos predecidos en test 60.22


## Modelo 4:  Logistic regression

Es una version similar a las anteriores con Y binaria.

In [91]:
# Modelo Logistic

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    penalty='l1',
    solver='saga',  # or 'liblinear'
    C=.001)

model=model.fit(X_train, y_train)
partidos_train=model.score(X_train,y_train)
partidos_test=model.score(X_test,y_test)

print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))

% de partidos predecidos en train 56.46
% de partidos predecidos en test 56.27


/Users/marcoyel21/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Modelo 4:  RF

Es una version similar a las anteriores con Y binaria. La ventaja de este modelo esque es muy poderoso y con tan pocos  datos y preprocesamiento lgora un desempeño tremendo. Muy probablemente nos quedemos con una version del RF o de un XGBoosting.

In [75]:
### RF model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)


partidos_train=clf.score(X_train,y_train)
partidos_test=clf.score(X_test,y_test)

print('% de partidos predecidos en train', round(partidos_train*100,2))
print('% de partidos predecidos en test', round(partidos_test*100,2))

% de partidos predecidos en train 67.46
% de partidos predecidos en test 64.01


# Conclusion

Probablemente nos quedemos con un RF o un XBoosting debido a la flexibilidad de los inputs y poder predictivo. Sin embargo, este ejercicio nos ha servido para proponer metricas ad hoc a nuestro problema; extensiones de los modelos que se las podriamos agregar a cualquier versión que elijamos;para seleccionar mejor los datos que vamos a usar; y pensar sobre la manera en que se van a usar los modelos en producción, es decir ¿qué tipo de datos se van a tener antes de un partido?